 ### 데이터베이스
  * 자료를 체계적으로 정리하여 저장하여 다수의 사용자에게 공유한 공용데이터
  * DBMS(데이터베이스 관리 시스템) --> MariaDB(MySQL), ORACLE, SQLITE
  * 데이터를 생성, 수정, 관리(CRUD)
  * SQL문 사용

10.1 관계형데이터베이스
  * 행,열로 이루어진 2차원 테이블

  * (1) DBMS(데이터베이스 관리 시스템) --> MariaDB(MySQL), ORACLE, SQLITE
  * (2) 기능
    * 정의(Definition): 자료 형태, 구조 정의
    * 조작(Manipulation): 수정,삽입,삭제
    * 제어(Control): 데이터 베이스의 정확성과 안정성 유지(무결성, 보안, 제어등의 기능)

10.2 SQLite3
  * 소형 DMBS
  * 외부공유가 안되고 기기 내부에서만 사용
  * python에 내장되어 있음(import sqlite3)
  *
  * 10.2.1 테이블 만들기
  ```sqlite3
      Create table [if not exists] table-name("name" "type" "constraint")
  ```

  * 10.2.2 CRUD
    * Create - 생성 - insert
      * ```insert into table (column-name) values;```
    * Read - 읽기/조회 - select
      * ```Select column(또는 *) from table where 조건;```
    * Update - 수정/갱신 - update
      * ```update table set column = 수정값 where 조건식;```
    * Delete - 삭제 - delete
      * ```Delete from table-name where 조건식;```

### 실습1 - SQLite3 데이터베이스 만들기 예(289~290p)

In [33]:
# sqlite3(내장)
import sqlite3
print("sqllit3 버전: ", sqlite3.sqlite_version_info)  # (3,33,0)  버전확인


try:
    # db 생성 및 연결 (sqlite3.connect)
    conn = sqlite3.connect("../DATA/sqlite_db")

    # sql문 실행 객체 (sqlite3.connect.cursor)
    cursor = conn.cursor()

    # table 생성 sql문 작성
    sql = 'create table if not exists test_table(name text(10), phone  text(15), addr text(50))'

    # sql문 실행
    cursor.execute(sql)

    # 레코드 추가
    cursor.execute("insert into test_table values('홍길동', '010-1111-1111', '서울시')")
    cursor.execute("insert into test_table values('이순신', '010-2222-2222', '해남시')")
    cursor.execute("insert into test_table values('강감찬', '010-1111-1111', '평양시')")

    # 레코드 조회
    cursor.execute("select * from test_table")  # 출력은 cursor에 들어가 있음
    rows = cursor.fetchall()    # fetchall을 이용해 모든 rows의 조회값 반환

    # 이쁘게 출력
    print('이름 \t전화번호  \t주소')
    for row in rows:
        print(row[0], '\t', row[1], '\t', row[2])

except Exception as e:
    print('db 연동 실패 : ', e)
    conn.rollback() # 실행취소
    # Roll back to the start of any pending transaction. If there is no open transaction, this method is a no-op.

finally:
    cursor.close()  # cursor 닫기
    conn.close()    # conn 닫기

sqllit3 버전:  (3, 33, 0)
이름 	전화번호  	주소
홍길동 	 010-1111-1111 	 서울시
이순신 	 010-2222-2222 	 해남시
강감찬 	 010-1111-1111 	 평양시


### 실습2 - CRUD문으로 테이블 조작하기(291~292p)

In [31]:
import sqlite3

try:
    # db 생성 및 연결 (sqlite3.connect)
    conn = sqlite3.connect("../DATA/sqlite_db")

    # sql문 실행 객체 (sqlite3.connect.cursor)
    cursor = conn.cursor()

    # goods table 생성
    sql = """ create table if not exists goods(
    code integer primary key,
    name text(30) unique not null,
    su integer  default 0,
    dan real default 0.0
    )"""
    cursor.execute(sql)

    # 레코드 추가
    cursor.execute("insert into goods values(1, '냉장고', 2, 8500000)")
    cursor.execute("insert into goods values(2, '세탁기', 3, 5500000)")
    cursor.execute("insert into goods(code, name) values(3, '전자레인지')")
    cursor.execute("insert into goods(code, name, dan) values(4, 'HDTV', 15000000)")
    conn.commit() # db 반영

    # 레코드 전체 상품 조회
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()

    print('검색된 레코드 수 : ', len(rows))
    for row in rows:
        print(row[0], '\t', row[1], '\t', row[2], '\t', row[3])
    

    # 외부 아규먼트로 원하는 상품명 조회
    name = input("조회할 상품명 입력 : ")
    sql = f"select * from goods where name like '%{name}'"
    cursor.execute(sql)
    rows = cursor.fetchall()

    if rows: 
        for row in rows:
            print(row)
    else:
        print("검색된 레코드 없음")

except Exception as e:
    print("db 연동 error : ", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()

검색된 레코드 수 :  4
1 	 냉장고 	 2 	 8500000.0
2 	 세탁기 	 3 	 5500000.0
3 	 전자레인지 	 0 	 0.0
4 	 HDTV 	 0 	 15000000.0
(1, '냉장고', 2, 8500000.0)


### 실습3 - CRUD문으로 테이블 조작하기(293~294p)
* 키보드로 입력받은 값을 레코드로 삽입하는 sql문
* 상품코드(code)를 기준으로 수량과 단가를 수정하는 sql문
* 상품코드(code)를 기준으로 레코드를 삭제하는 sql문

In [34]:
import sqlite3

try:
    # db 생성 및 연결 (sqlite3.connect)
    conn = sqlite3.connect("../DATA/sqlite_db")

    # sql문 실행 객체 (sqlite3.connect.cursor)
    cursor = conn.cursor()

    # table 생성
    sql = """ create table if not exists goods(
    code integer primary key,
    name text(30) unique not null,
    su integer  default 0,
    dan real default 0.0
    )"""
    cursor.execute(sql)

    #print("111111111111111")

    # 레코드 추가-1
    cursor.execute("insert into goods values(1, '냉장고', 2, 8500000)")
    cursor.execute("insert into goods values(2, '세탁기', 3, 5500000)")
    cursor.execute("insert into goods(code, name) values(3, '전자레인지')")
    cursor.execute("insert into goods(code, name, dan) values(4, 'HDTV', 15000000)")
    conn.commit() # db 반영

    #print("111111111111111")

    # 키보드로 레코드 추가-2
    code = int(input('code 입력 : '))
    name = input('name 입력 : ')
    su = int(input('su 입력 : '))
    dan = int(input('dan 입력 : '))
    
    sql = f"insert into goods values({code}, '{name}', {su}, {dan})"
    cursor.execute(sql)
    conn.commit()

    # 레코드 수정 : code --> su, dan 수정
    ''' 주석처리
    code = int(input('수정 code 입력 : '))
    su = int(input('수정 su 입력 : '))
    dan = int(input('수정 dan 입력 : '))

    sql = f"insert into goods values({code}, '{name}', {su}, {dan})"
    cursor.execute(sql)
    conn.commit()
    '''

    # 레코드 삭제 : code --> 삭제
    ''' 주석처리
    code = int(input('삭제 code 입력 : '))

    sql = f"delete from goods where code = {code}"
    cursor.execute(sql)
    conn.commit()
    '''

    #print("111111111111111")

    # 레코드 조회
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()

    for row in rows:
        print(row[0], '\t', row[1], '\t', row[2])


    # 상품명 조회
    name = input("조회 상품명 입력 : ")
    sql = f"select * from goods where name like '%{name}%'"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    if rows: 
        for row in rows:
            print(row)
    else:
        print("검색된 레코드 없음")

except Exception as e:
    print("db 연동 error : ", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()

1 	 냉장고 	 2
2 	 세탁기 	 3
3 	 전자레인지 	 0
4 	 HDTV 	 0
5 	 가습기 	 3
(5, '가습기', 3, 200000.0)
